In [1]:
import os
import numpy as np
import torch
from PIL import Image
from pydrive.drive import GoogleDrive
from google.colab import drive
from pydrive.auth import GoogleAuth
from google.colab import auth
import torchvision.transforms as transforms
from oauth2client.client import GoogleCredentials
drive.mount('/content/drive')
# BASICS 
import os
import numpy as np
import torch

# Colab 
from pydrive.drive import GoogleDrive
from google.colab import drive
from google.colab import files
from pydrive.auth import GoogleAuth
from google.colab import auth
import torchvision.transforms as transforms
from oauth2client.client import GoogleCredentials
drive.mount('/content/drive')
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)
# MODEL and TRAIN
import torchvision
import torchvision.models
from torch.utils.data import DataLoader,Dataset
import random
from skimage import io
import torch.nn as nn
import torch.nn.functional as F
from matplotlib import pyplot as plt
from torch import optim
from PIL import Image

device='cuda:0'

Mounted at /content/drive
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
def dice( inputs, targets):
    """dice coefficient calculation function.
        intersection*2/pixelnumber(Inputs)+pixelnumber(targets)

    :param input  : the output of the model.
    :param targets: The reference mask.

    :returns : dice coefficent  .
    :rtype   : float
    """

    # Input can be values rather then 0 and 1 to arrange it
    inputs = torch.where(inputs>=0.5, 1, 0)
    inputs = torch.flatten(inputs)
    targets = torch.flatten(targets)
    # The the part where we have calcualted the function 
    dice = (1e-5+torch.sum(inputs*targets)*2.0) / (torch.sum(targets) + torch.sum(inputs))
    return float(dice)
def iou(y_true, y_pred, smooth = 0.1):
    y_true= torch.where(y_true>=0.5, 1, 0)
    intersection = torch.sum(y_true * y_pred)
    sum_ = torch.sum(y_true) + torch.sum(y_pred)
    jac = (intersection + smooth) / (sum_ - intersection + smooth)
    return jac


def binary_acc(y_true, y_pred, smooth = 0.1):
    y_true= torch.where(y_true>=0.5, 1, 0)
    y_true=torch.flatten(y_true)
    y_pred=torch.flatten(y_pred)
    sum_ = torch.sum(y_true==y_pred)
    acc = ( sum_ ) / (y_pred.shape[0])
    return acc


In [3]:
%%shell

# Download TorchVision repo to use some files from
# references/detection
git clone https://github.com/pytorch/vision.git
cd vision
git checkout v0.8.2

cp references/detection/utils.py ../
cp references/detection/transforms.py ../
cp references/detection/coco_eval.py ../
cp references/detection/engine.py ../
cp references/detection/coco_utils.py ../

Cloning into 'vision'...
remote: Enumerating objects: 153845, done.
remote: Counting objects: 100% (6218/6218), done.
remote: Compressing objects: 100% (629/629), done.
remote: Total 153845 (delta 5660), reused 6056 (delta 5563), pack-reused 147627
Receiving objects: 100% (153845/153845), 301.34 MiB | 11.69 MiB/s, done.
Resolving deltas: 100% (136253/136253), done.
Note: checking out 'v0.8.2'.

You are in 'detached HEAD' state. You can look around, make experimental
changes and commit them, and you can discard any commits you make in this
state without impacting any branches by performing another checkout.

If you want to create a new branch to retain commits you create, you may
do so (now or later) by using -b with the checkout command again. Example:

  git checkout -b <new-branch-name>

HEAD is now at 2f40a483d [v0.8.X] .circleci: Add Python 3.9 to CI (#3063)


In [4]:

from engine import train_one_epoch, evaluate
import utils
import transforms as T

In [5]:
# !cp /content/drive/MyDrive/Covid/functions.ipynb  /content  

In [6]:
# %run functions.ipynb

In [7]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [8]:
from torchvision.models.resnet import resnet101

In [9]:
class kaan_dataset(Dataset): # inherit from torch.utils.data.Dataset
    "Lung and covid-19 sengmentation dataset."

    def __init__(self,root_dir = ("/content/archive"),class_folder='lung',split = "Train", transforms = None , shuffle = True,batch_size=1,
                 sample_size=500 ):
        """
        Args:
        :param root_dir (str): the location of the datasetfolders 
        :param class_folder (str): the lung or covid default to lung 
        :param split (str): the usage purpose of the data
        :param transforms( optional): for normalization  ,default to None
        :param batch_size (int, optional) : default 1 
        :param class (str) : lung or covid : to call the dataset, default to lung:  
        :param sample_size (int) : default to 500
        """

        self.root_dir = root_dir
        self.class_folder=class_folder
        self.split = split # Test Train Val
        self.transforms = transforms
        self.batch_size=batch_size
        self.sample_size=sample_size

        # used to store image and mask file names 
        image_file=[]
        mask_file=[]
        mask_folder=[]
        array_file=[]
        # Label array used to reach folders 
        self.label_array=["/COVID-19","/Non-COVID","/Normal"]
        self.image_folder='/images/'
        # If the dataset called for lung segmentattion  clas folder will be acording to that 
        if class_folder=='lung':
          self.class_folder='/Lung Segmentation Data/Lung Segmentation Data/'
          self.mask_folder='/lung masks/'
        else:
          self.class_folder='/Infection Segmentation Data/Infection Segmentation Data/'
          self.mask_folder='/infection masks/'
        
        # loop iterated 3 times for labels and the data filled with the form of the image ans mask as 
        """array_file example data: /COVID-19/images/covid123.png """
        # Image label/folder/file
        for label_num in range(len(self.label_array)):
          array_file=[]
          path_image=self.root_dir+self.class_folder+self.split+self.label_array[label_num]+self.image_folder
          path_mask=self.root_dir+self.class_folder+self.split+self.label_array[label_num]+self.mask_folder
          # Flatten name_list 
          name_list_image=(os.listdir(path_image))
          name_list_mask=(os.listdir(path_mask))
          data_image=[fName for fName in name_list_image ]
          data_mask=[fName for fName in name_list_mask ]
          # arrange the locations of the images and mask on the right form 
          for x in range(len(data_image)):
            array_file.append('{}{}{}'.format(self.label_array[label_num],self.image_folder,data_image[x]))
          image_file.append(array_file)
          array_file=[]
          for x in range(len(data_mask)):
            array_file.append('{}{}{}'.format(self.label_array[label_num],self.mask_folder,data_mask[x]))
          mask_file.append(array_file)
        # flatten and shorten the size of the dataset 
        self.image_file=[item for sublist in image_file for item in sublist][:sample_size]  
        self.mask_file=[item for sublist in mask_file for item in sublist][:sample_size]  
        self.mask_idx = [ind for ind in range(sample_size) ]
        # Shuffle the dataset 
        if split=='Train' and shuffle==True:
          c = list(zip(self.image_file, self.mask_file,self.mask_idx))
          random.shuffle(c)
          self.image_file, self.mask_file,self.mask_idx = zip(*c)
        

        # Shorten the files after shuffle 
        self.image_file=self.image_file[:sample_size]
        self.mask_file=self.mask_file[:sample_size]
        self.mask_idx=self.mask_idx[:sample_size]
        # Storig the data to the dictionary
        self.data_file = {"Data"  : {"image":self.image_file, "mask": self.mask_file}}
        self.mask_idx ={"idx" : self.mask_idx}
        self.device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

    def __len__(self):
        return len(self.mask_idx['idx'])

    def __getitem__(self, idx):
        idx = self.mask_idx['idx'][idx]
        # set index
        # The paths of the images and the masks have called
        img_path=os.path.join(self.root_dir+self.class_folder+self.split+self.data_file['Data']['image'][idx])
        mask_path=os.path.join(self.root_dir+self.class_folder+self.split+self.data_file["Data"]['mask'][idx])

        # Colling the path to reach the images and the masks 
        # The critical part in here is to use 'L' because it helps us to use the image its non int values 
        # For the mask files the images do not  need to be in form of that.  
        img  = Image.open(img_path).convert('L')
        mask = Image.open(mask_path)
        # print(img)
        # print(img_path)
        trans1 = transforms.ToTensor()
        #trans2= transforms.ToTensor()

        img=trans1(img)
        #mask=trans2(mask)
        # The return dictionary 
        mask = np.array(mask)
        # instances are encoded as different colors
        obj_ids = np.unique(mask)
        # first id is the background, so remove it
        obj_ids = obj_ids[1:]
        #print(obj_ids)
        # split the color-encoded mask into a set
        # of binary masks
        masks = mask == obj_ids[:, None, None]

        # get bounding box coordinates for each mask
        num_objs = len(obj_ids)
        boxes = []
        for i in range(num_objs):
            pos = np.where(masks[i])
            xmin = np.min(pos[1])
            xmax = np.max(pos[1])
            ymin = np.min(pos[0])
            ymax = np.max(pos[0])
            boxes.append([xmin, ymin, xmax, ymax])

        # convert everything into a torch.Tensor
        boxes = torch.as_tensor(boxes, dtype=torch.float32)
        # there is only one class
        labels = torch.ones((num_objs,), dtype=torch.int64)
        masks = torch.as_tensor(masks, dtype=torch.uint8)

        image_id = torch.tensor([idx])
        area = (boxes[:, 3] - boxes[:, 1]) * (boxes[:, 2] - boxes[:, 0])
        # suppose all instances are not crowd
        iscrowd = torch.zeros((num_objs,), dtype=torch.int64)

        target = {}
        target["boxes"] = boxes
        target["labels"] = labels
        target["masks"] = masks
        target["image_id"] = image_id
        target["area"] = area
        target["iscrowd"] = iscrowd
        sample = {'image': img, 'mask': mask}
       
        # Changing the dtype and form of the sample data
        # 
        # sample=trans1(sample)
        if self.transforms:
            sample = self.transforms(img,target)

        
        return img,target

In [10]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [11]:
# The Calling the validaton dataset
val=kaan_dataset(split = "Val",class_folder='covid',shuffle=False,sample_size=1000)# The length of the validation dataset to calculate the average value of the loss and accuracy

val_size=val.__len__()
# The Train dataset 
tra=kaan_dataset(split = "Train",class_folder='covid',shuffle=False,sample_size=5000)
tra_size=tra.__len__()

# The Train dataset 
Test=kaan_dataset(split = "Test",class_folder='covid',shuffle=False)
Test_size=Test.__len__()

In [12]:
import torchvision
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor

# load a model pre-trained on COCO
model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=True)

# replace the classifier with a new one, that has
# num_classes which is user-defined
num_classes = 2  # 1 class (person) + background
# get number of input features for the classifier
in_features = model.roi_heads.box_predictor.cls_score.in_features
# replace the pre-trained head with a new one
model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)

Downloading: "https://download.pytorch.org/models/fasterrcnn_resnet50_fpn_coco-258fb6c6.pth" to /root/.cache/torch/hub/checkpoints/fasterrcnn_resnet50_fpn_coco-258fb6c6.pth


  0%|          | 0.00/160M [00:00<?, ?B/s]

In [13]:
batch_size=12
# loading images and masks for mini epochs 
data_loader = torch.utils.data.DataLoader(
        tra, batch_size=batch_size,  num_workers=2, drop_last=True, collate_fn=utils.collate_fn)

data_loader_val = torch.utils.data.DataLoader(
        val, batch_size=batch_size,  num_workers=2, drop_last=True, collate_fn=utils.collate_fn)

data_loader_Test = torch.utils.data.DataLoader(
        Test, batch_size=batch_size,  num_workers=2, drop_last=True, collate_fn=utils.collate_fn)




In [14]:
import torchvision
from torchvision.models.detection import FasterRCNN
from torchvision.models.detection.rpn import AnchorGenerator
import torchvision
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from torchvision.models.detection.mask_rcnn import MaskRCNNPredictor
# load a pre-trained model for classification and return
# only the features
backbone = torchvision.models.mobilenet_v2(pretrained=True).features
# FasterRCNN needs to know the number of
# output channels in a backbone. For mobilenet_v2, it's 1280
# so we need to add it here
backbone.out_channels = 1280

# let's make the RPN generate 5 x 3 anchors per spatial
# location, with 5 different sizes and 3 different aspect
# ratios. We have a Tuple[Tuple[int]] because each feature
# map could potentially have different sizes and
# aspect ratios
anchor_generator = AnchorGenerator(sizes=((32, 64, 128, 256, 512),),
                                   aspect_ratios=((0.5, 1.0, 2.0),))

# let's define what are the feature maps that we will
# use to perform the region of interest cropping, as well as
# the size of the crop after rescaling.
# if your backbone returns a Tensor, featmap_names is expected to
# be [0]. More generally, the backbone should return an
# OrderedDict[Tensor], and in featmap_names you can choose which
# feature maps to use.
roi_pooler = torchvision.ops.MultiScaleRoIAlign(featmap_names=['0'],
                                                output_size=7,
                                                sampling_ratio=2)

# put the pieces together inside a FasterRCNN model
model = FasterRCNN(backbone,
                   num_classes=1,
                   rpn_anchor_generator=anchor_generator,
                   box_roi_pool=roi_pooler)



def get_model_instance_segmentation(num_classes):
    # load an instance segmentation model pre-trained on COCO
    model = torchvision.models.detection.maskrcnn_resnet50_fpn(pretrained=True)

    # get number of input features for the classifier
    in_features = model.roi_heads.box_predictor.cls_score.in_features
    # replace the pre-trained head with a new one
    model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)

    # now get the number of input features for the mask classifier
    in_features_mask = model.roi_heads.mask_predictor.conv5_mask.in_channels
    hidden_layer = 256
    # and replace the mask predictor with a new one
    model.roi_heads.mask_predictor = MaskRCNNPredictor(in_features_mask,
                                                       hidden_layer,
                                                       num_classes)

    return model
  

Downloading: "https://download.pytorch.org/models/mobilenet_v2-b0353104.pth" to /root/.cache/torch/hub/checkpoints/mobilenet_v2-b0353104.pth


  0%|          | 0.00/13.6M [00:00<?, ?B/s]

In [15]:
#device='cpu'
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
#device = torch.device('cpu')

model = get_model_instance_segmentation(2).to(device)

# move model to the right device

model.to(device)
  # construct an optimizer
params = [p for p in model.parameters() if p.requires_grad]
optimizer = torch.optim.Adam(params, lr=0.0005)
# and a learning rate scheduler
lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer,
                                                step_size=3,
                                                gamma=0.1)

# let's train it for 10 epochs
num_epochs = 10
trans1 = transforms.ToTensor()
trans2 = transforms.ToTensor()

Downloading: "https://download.pytorch.org/models/maskrcnn_resnet50_fpn_coco-bf2d0c1e.pth" to /root/.cache/torch/hub/checkpoints/maskrcnn_resnet50_fpn_coco-bf2d0c1e.pth


  0%|          | 0.00/170M [00:00<?, ?B/s]

In [16]:
for x in range(30):
  train_one_epoch(model, optimizer, data_loader, device, x, 15)
  evaluate(model, data_loader, device)


Epoch: [0]  [ 0/50]  eta: 0:16:19  lr: 0.000011  loss: 5.3667 (5.3667)  loss_classifier: 0.8636 (0.8636)  loss_box_reg: 0.1701 (0.1701)  loss_mask: 4.3221 (4.3221)  loss_objectness: 0.0030 (0.0030)  loss_rpn_box_reg: 0.0079 (0.0079)  time: 19.5951  data: 17.5561  max mem: 8531
Epoch: [0]  [15/50]  eta: 0:05:08  lr: 0.000164  loss: 0.8414 (1.6608)  loss_classifier: 0.0756 (0.2546)  loss_box_reg: 0.1020 (0.1136)  loss_mask: 0.6521 (1.2714)  loss_objectness: 0.0063 (0.0157)  loss_rpn_box_reg: 0.0040 (0.0055)  time: 8.8079  data: 7.3526  max mem: 8971
Epoch: [0]  [30/50]  eta: 0:02:57  lr: 0.000317  loss: 0.4329 (1.0569)  loss_classifier: 0.0426 (0.1511)  loss_box_reg: 0.0868 (0.1000)  loss_mask: 0.2810 (0.7907)  loss_objectness: 0.0046 (0.0109)  loss_rpn_box_reg: 0.0031 (0.0042)  time: 8.5377  data: 7.1167  max mem: 8971
Epoch: [0]  [45/50]  eta: 0:00:43  lr: 0.000469  loss: 0.2130 (0.7807)  loss_classifier: 0.0228 (0.1091)  loss_box_reg: 0.0559 (0.0853)  loss_mask: 0.1288 (0.5746)  loss_

In [17]:
evaluate(model, data_loader_Test, device)

creating index...
index created!
Test:  [ 0/41]  eta: 0:00:36  model_time: 0.6236 (0.6236)  evaluator_time: 0.0162 (0.0162)  time: 0.8953  data: 0.2524  max mem: 8971
Test:  [40/41]  eta: 0:00:00  model_time: 0.5986 (0.5992)  evaluator_time: 0.0141 (0.0145)  time: 0.6564  data: 0.0404  max mem: 8971
Test: Total time: 0:00:27 (0.6608 s / it)
Averaged stats: model_time: 0.5986 (0.5992)  evaluator_time: 0.0141 (0.0145)
Accumulating evaluation results...
DONE (t=0.03s).
Accumulating evaluation results...
DONE (t=0.04s).
IoU metric: bbox
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.848
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 1.000
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.990
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = -1.000
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = -1.000
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | m